# Cours : Complexité

## Introduction
De manière générale, un algorithme effectue un traitement sur des données.
S'il produit le résultat escompté, on dit que l'algorithme est *correct*.
Mais cela ne suffit pas. On veut que l'algorithme soit *efficace*, c'est-à-dire
que le *temps de calcul* du résultat soit raisonnable.

## Plusieurs chemins mènent à Rome, mais...

Il y a souvent plusieurs façons d'arriver au résultat visé, autrement dit,
plusieurs algorithmes pour résoudre un problème.

On veut, par exemple, déplacer le premier élément d'un tableau pour le mettre à la fin.
C'est ce qu'on appelle un *décalage circulaire*.

**Exemple**
* tableau de départ : `[1, 2, 3, 4, 5]` 

après le *décalage circulaire*

* résultat : `[2, 3, 4, 5, 1]`

### Première solution
1. Échanger les 1er et 2e éléments,
2. échanger les 2e et 3e,
3. échanger les 3e et 4e,
4. et ainsi de suite jusqu'au dernier élément.

In [5]:
def decalage1(t):
    i = 0
    while i < len(t)-1:
        tmp = t[i]
        t[i] = t[i+1]
        t[i+1] = tmp
        i += 1
    
tab = [1,2,3,4,5]
decalage1(tab)
print(tab)

[2, 3, 4, 5, 1]


### Deuxième solution
1. Sauvegarder le premier élément du tableau dans une variable,
2. décaler tous les éléments d'une case vers la gauche,
3. écrire l'élément sauvegardé en dernière position.

In [6]:
def decalage2(t):
    tmp = t[0]
    i = 0
    while i < len(t)-1:
        t[i] = t[i+1]
        i += 1
    t[len(t)-1] = tmp

tab = [1,2,3,4,5]
decalage2(tab)
print(tab)

[2, 3, 4, 5, 1]


Les deux approches sont correctes ; cependant, dans le cadre de ce cours, la question que l'on se pose est  
<center>"Lequel est le plus rapide ?"</center>

## Mesure des performances avec `time`

Pour répondre de manière expérimentale à la question précédente, on peut comparer le temps d'exécution nécessaire à chaque algorithme pour des mêmes données en entrée.

Pour cela, on crée une fonction `gen_tab_alea` qui retourne un tableau d'entiers aléatoires. Cette fonction permettra de fournir des données en entrée de chaque algorithme.

In [7]:
from random import random

def gen_tab_alea(n):
    tab = []
    i = 0
    while i < n:
        tab.append(random())
        i += 1
    return tab

t = gen_tab_alea(4)
print(t)

[0.9698918888844585, 0.2272125529478758, 0.0768574910044102, 0.7216411436103358]


La bibliothèque `time` fournit un chronomètre (fonction `time`) qui permet de mesurer le temps d'exécution d'un code. Pour utiliser ce chronomètre, il faut importer cette fonction :

In [11]:
from time import time

Pour mesurer le temps d'exécution d'un code, il faut
* lancer le chrononomètre au début du programme et l'arrêter à la fin.
* calculer ensuite la différence entre les temps de départ et d'arrivée (pour obtenir la durée en sencondes).

On peut chronométrer, par exemple, le temps nécessaire pour effectuer un décalage circulaire sur un tableau de taille 10000 avec la première méthode.

In [36]:
tab = gen_tab_alea(10000)

tic = time()       # top départ

########### Début du traitement à chronométer ###########
decalage1(tab)
############ Fin du traitement à chronométer ############

tac = time()           # arrêt du chronomètre

print(round(1000*(tac-tic),2)," ms")     # on convertit en ms et on arrondit au centième

1.72  ms


En exécutant plusieurs fois ce programme, on constate que le temps n'est pas toujours
le même. En effet, plusieurs processus sont actifs en permanence sur la machine (applications en tâche de fond, interface graphique, réseau...) et peuvent mobiliser le processeur. Pour atténuer ce phénomène, on peut calculer la moyenne d'un certain nombre de mesures (par exemple 500) des temps d'exécution.

In [9]:
def mesure_decalage1(tab):

    nb_mesures = 500
    
    tic = time()       # top départ
    
    # On fait nb_messures decalages circulaires avec decalage1
    i = 0
    while i < nb_mesures:
        decalage1(tab)
        i+=1
    
    tac = time()        
    # arrêt du chronomètre
    
    # On fait la moyenne des temps d'exécutions (ms) des nb_mesures exécutions et on arrondit au millième
    return round(1000*(tac-tic) / nb_mesures, 3)



# Fonction pour mesurer le temps moyen pour effectuer un décalage circulaire avec la deuxième méthode
def mesure_decalage2(tab):
    nb_mesures = 500    
    tic = time()
    i = 0
    while i < nb_mesures:
        decalage2(tab)
        i+=1
    tac = time()
    return round(1000*(tac-tic) / nb_mesures, 3)


n = 10000
tab1 = gen_tab_alea(n)
tab2 = tab1.copy()
tps = mesure_decalage1(tab1)
print("Tps moyen pour un décalage circulaire sur un tableau de taille", n," avec decalage1 :", tps, "ms")
tps = mesure_decalage2(tab2)
print("Tps moyen pour un décalage circulaire sur un tableau de taille", n, " avec decalage2 :", tps, "ms")

NameError: name 'time' is not defined

En regardant les temps d'exécution, on s'aperçoit la deuxième méthode pour le décalage circulaire est plus rapide que la première méthode.

## Opérations élémentaires et complexité

Il est très difficile de comparer expérimentalement deux algorithmes en mesurant leur temps d'exécution, comme dans l'exemple ci-dessus. Il faut en effet implémenter les deux algorithmes et faire de nombreux tests sur le même ordinateur (la vitesse du processeur, cache, accès disque variant beaucoup d'un ordinateur à un autre). 

La théorie de la complexité permet de comparer des algorithmes sans même les implémenter. Elle permet aussi d'avoir une idée précise sur le comportement d'un algorithme suivant les données en entrée. Par exemple, elle permet de répondre à des questions du type : "si le tableau en entrée est 100 fois plus grand, est-ce que l'algorithme sera 100 fois plus long ? 10000 fois plus long, aussi long ?"

La théorie de la complexité est basée sur le calcul du nombre d'opérations élémentaires. Une *opération élémentaire* correspond à une opération simple : affectation, addition, soustraction, multiplication, division, comparaison de valeurs, etc. On suppose qu'un ordinateur met le même temps pour faire n'importe quelle opération (petite approximation). *Pour avoir une idée de la durée d'exécution, il suffit de connaître le nombre d'opérations élémentaires nécessaires à effectuer ainsi que le temps nécessaire pour effectuer une opération élémentaire.*

### Nombre d'opérations élémentaires pour le décalage circulaire

#### Première méthode

On a autant d'itérations qu'il y a de valeurs dans le tableau. À chaque itération, on a comme opérations élémentaires :
* $4$ affectations,
* $3$ additions,
* $1$ soustraction,
* $1$ comparaison.

On a donc $9$ opérations élémentaires par itération. On a aussi une affectation au départ (`i = 0`). Si le tableau a $n$ cases, alors le nombre d'opérations élémentaires est : $9n + 1$.

#### Deuxième méthode

On a autant d'itérations qu'il y a de valeurs dans le tableau. À chaque itération, on a comme opérations élémentaires :
* $2$ affectations,
* $2$ additions,
* $1$ soustraction,
* $1$ comparaison.

On a donc $6$ opérations élémentaires par itération. On a aussi deux affectations et une soustraction en dehors de la boucle. Si le tableau a $n$ cases, alors le nombre d'opérations élémentaires est : $6n + 3$.

#### Comparaison

Si le tableau fait 10000 cases, le premier algorithme nécessite 90001 opérations alors que le deuxième nécessite 60003 opérations élémentaires. Théoriquement, pour un tableau de taille 10000, le deuxième algorithme est environ 1.5 fois plus rapide (90001/60003). On retrouve un résultat similaire expérimentalement en comparant les temps d'exécution. 

#### Estimation du temps d'exécution pour un tableau de taille 100000

Si la taille du tableau est multipliée par 10, alors le nombre d'opérations élémentaires est aussi multiplié par 10 et l'algorithme prendra alors 10 fois plus de temps. On peut ainsi estimer le temps d'exécution nécessaire pour des très grandes données sans tester expérimentalement.

**Sur ce thème :** Exercice 1 du TD.

## Complexité dans le pire des cas

En général, le nombre d'opérations dépend non seulement de la taille des données,
mais des données elles-mêmes (on dit aussi : de l'instance).

Par exemple, si l'on recherche un élément particulier dans un tableau de taille `n`, avec de la chance, l'élément est trouvé du premier coup et le programme s'arrête après une opération.

Mais il est intéressant de prévoir la performance pour un jeu de données
défavorable ou la **complexité dans le pire des cas**. Dans l'exemple de la recherche,
le pire des cas se produit quand l'élément n'est pas trouvé ou trouvé en dernier,
ce qui oblige à parcourir le tableau entier. 

## Complexité asymptotique


Calculer le nombre exact d'opérations élémentaires peut s'avérer difficile et n'est pas nécessaire, une estimation suffit. 

Le nombre d'opérations élémentaires peut être vu comme une fonction mathématique dépendant de la taille de l'instance. Ainsi, les complexités des méthodes de décalage sont les fonctions $f(n) = 9n +1$ et $f(n) = 6n + 3$. La complexité asymptotique consiste à comparer la fonction de complexité avec des fonctions de référence (puissance (linéaire, quadratique, etc), logarithmique, exponentielle, etc) pour des instances de grande taille (quand $n$ est grand). 

Par exemple : 
* La complexité d'un algorithme est **linéaire** si le nombre d'opérations élémentaires est environ $Cn$ où $C$ est une constante,
* La complexité d'un algorithme est **quadratique** si le nombre d'opérations élémentaires est environ $Cn^2$ où $C$ est une constante,
* La complexité d'un algorithme est **logarithmique** si le nombre d'opérations élémentaires est environ $C\log_2(n)$ où $C$ est une constante.

La complexité d'un algorithme est notée $O(g(n))$ (prononcée *grand Ô de g(n)*) , où $g(n)$ est une fonction de référence. Une complexité linéaire est notée $O(n)$, une complexité quadratique $O(n^2)$, etc.

**Sur ce thème :** Exercices 2 et 3 du TD.

